<a href="https://colab.research.google.com/github/swetha4444/Artificial-Intelligence/blob/master/183_assign6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Assignment 6

Davis–Putnam–Logemann–Loveland (DPLL) algorithm
Implement David-Putnam algorithm for checking the satisfiability of a sentence represented in propositional logic.

```
Swetha Saseendran
CSE-C
185001183
```




In [81]:
class DPLL:

    true_literals = set()
    false_literals = set()
    up,pr,branches = 0,0,0
    clauses = []
    literals = []



    def __init__(self, formula):
      #extracting all literals from all clauses
      self.literals = [literal for literal in list(set(formula)) if literal.isalpha()]
      #extracting all clauses from the formula
      self.clauses = formula.split(", ")
      if (len(self.literals) == 0):
        self.clauses = []



    def __str__(self):
        f_str = ""
        i = 0
        if (len(self.clauses) == 0):   #formula is empty
            f_str = "()"
        else:
            for clause in self.clauses:  #represent each claue in CNF
                if (len(clause) > 0):
                    f_str += '(' + clause.replace(' ', ' ∨ ')
                    i += 1
                if (i <= len(self.clauses)-1):
                    f_str += ') ∧ '
                else:
                    f_str += ')'
        return f_str


    def pure_lit(self):
      clauses = list(set(self.clauses)) #unique clauses
      unit_clauses = [clause for clause in clauses if len(clause) < 3] #implies X or ¬X
      unit_clauses = list(set(unit_clauses)) #unique unit clauses
      pure_literals = set()
      flag,i = 0,0
      self.pr += 1
      
      print("CNF before removing Pure Literals:\t", self)
      #removing clauses with all pure literals
      for unit_clause in self.literals:
        print("\tChecking for: ",unit_clause)
        for i in range(0,len(clauses)): #checking if its a pure literal
          if(('¬' + unit_clause) not in clauses[i]):
            flag = 1
          else:
            flag = 0
         #is pure literal
        if (flag == 1):
          i = 0
          print("\t\t",unit_clause," is a pure literal")
          self.true_literals.add(unit_clause)
          pure_literals.add(unit_clause)
          print("Len: ",clauses)
          while(True):
            if (i >= len(clauses)):
              break
            if (unit_clause in clauses[i]):  #Remove the unit clause from the formula
              print("\t\t\tRemoving: ",clauses[i])
              clauses.remove(clauses[i]) #Entire clause becomes True hence no sig in entire cnf
              i -= 1
            i += 1
        else:
          print("\t\t",unit_clause," is not a pure literal")

      self.clauses = clauses
      print("Current Pure Literals:",list(pure_literals))
      print("CNF after removing Pure Literals:", self)
      return clauses



    def unit_prop(self):
      print("\nCurrent Formula:\t\t", self)      
       
      clauses = list(set(self.clauses)) #unique clauses
      unit_clauses = [clause for clause in clauses if len(clause) < 3] #implies X or ¬X
      unit_clauses = list(set(unit_clauses)) #unique unit clauses
      print("CNF before Unit Propagation:\t", self)
      print("Current Unit Clauses:\t", unit_clauses)

      if (len(unit_clauses) > 0):
        for unit_clause in unit_clauses:
          self.up += 1
          if '¬' in unit_clause: #¬X found
            #assign False to it
            self.false_literals.add(unit_clause[-1]) #adding literal
            i = 0
            while True:
              #unit subsumption
              if unit_clause in clauses[i]:  #Remove the unit clause from the formula
                clauses.remove(clauses[i])   #Entire clause becomes True hence no sig in entire cnf
                i -= 1
              #unit resolution
              elif unit_clause[-1] in clauses[i]:  #If a ¬X exists in the formula
                clauses[i] = clauses[i].replace(unit_clause[-1], '').strip() #V False => no sig in clause
              i += 1
              if (i >= len(clauses)):
                break
          else:  #X found
            self.true_literals.add(unit_clause)
            i = 0
            while True:
              #unit resolution
              if (('¬' + unit_clause) in clauses[i]):  #If a ¬X exists in the formula
                clauses[i] = clauses[i].replace('¬' + unit_clause, '').strip() #V False => no sig in clause
                if ('  ' in clauses[i]):  #Empty clause
                  clauses[i] = clauses[i].replace('  ', ' ')
              #unit subsumption
              elif unit_clause in clauses[i]:  #Remove the unit clause from the formula
                clauses.remove(clauses[i]) #Entire clause becomes True hence no sig in entire cnf
                i -= 1
              i += 1
              if (i >= len(clauses)):
                break
      self.clauses = clauses 
      print("CNF after Unit Propagation:\t", self)
      return self.clauses


    def DPLL_solver(self):
      self.branches += 1
      self.true_literals = set(self.true_literals)
      self.false_literals = set(self.false_literals)

      #Unit propogation
      clauses = self.unit_prop()
      
      if len(self.clauses) == 0:  #Formula becomes an empty clause
        return True

      
      self.literals = [literal for literal in list(set(''.join(self.clauses))) if literal.isalpha()] #form the literals again from the updated formula
      #Pure Literals
      clauses = self.pure_lit()

      if len(self.clauses) == 0:  #Formula becomes an empty clause
        return True

      if sum(len(clause) == 0 for clause in clauses):  #If there are empty parantheses, indicating contradiction
        for literal in new_true_literals:   #clear every newly set true literal
          self.true_literals.remove(literal)
        for literal in new_false_literals:  #clear every newly set false literal
          self.false_literals.remove(literal)
        print("\nNull clause found. \n\tBacktracking...")
        return False  #the formula is unsatisfiable.
        

      self.literals = [literal for literal in list(set(''.join(self.clauses))) if literal.isalpha()] #form the literals again from the updated formula
      first_literal = self.literals[0]  #choose the first literal to set as True/False to find satisfying assignments
      
      #Branch First
      print("\nTrying with {0} as True:-".format(first_literal))
      self.clauses  = clauses + [first_literal]  #Try with first literal set to True => just append the literal as a clause   
      if self.DPLL_solver():  #recursively work on the new formula
        self.branches += 1
        return True
      
      #Branch Second
      self.clauses = clauses + ['¬' + first_literal]  #Try with first literal set to False (i.e ¬(first literal) is True)
      print("\nTrying with ¬{0} as True:-".format(first_literal))
      if self.DPLL_solver():  #recursively work on the new formula
        self.branches += 1
        return True

      else:  #no satisfying assignments were found to the literals of the formula
        self.clauses = clauses
        for literal in new_true_literals:   #remove all the assignments
          self.true_literals.remove(literal)
        for literal in new_true_literals:   #remove all the assignments
          self.true_literals.remove(literal)
        return False  #the formula is unsatisfiable.


    def print_result(self, satisfiability):
        if satisfiability == True:
            print("\nThe given CNF statement is satisfiable.")
            print("\nAssignments: ")
            for literal in self.true_literals:
                print("\t"+literal, "= True") 
            for literal in self.false_literals:
                print("\t"+literal, "= False")  
        else:
            print("\nThe given CNF statement was unsatisfiable.") 

        print("\nNo. of branches:\t\t", self.branches)
        print("\nNo. of unit propagations:\t", self.up)
        print("\nNo. of pure literals recursions:", self.pr)
        

solver = DPLL("X Y Z, X ¬Y, ¬X Y ¬Z, ¬Z")
sat = solver.DPLL_solver()
solver.print_result(sat)







Current Formula:		 (X ∨ Y ∨ Z) ∧ (X ∨ ¬Y) ∧ (¬X ∨ Y ∨ ¬Z) ∧ (¬Z)
CNF before Unit Propagation:	 (X ∨ Y ∨ Z) ∧ (X ∨ ¬Y) ∧ (¬X ∨ Y ∨ ¬Z) ∧ (¬Z)
Current Unit Clauses:	 ['¬Z']
CNF after Unit Propagation:	 (X ∨ ¬Y) ∧ (X ∨ Y)
CNF before removing Pure Literals:	 (X ∨ ¬Y) ∧ (X ∨ Y)
	Checking for:  Y
		 Y  is not a pure literal
	Checking for:  X
		 X  is a pure literal
Len:  ['X Y', 'X ¬Y']
			Removing:  X Y
			Removing:  X ¬Y
Current Pure Literals: ['X']
CNF after removing Pure Literals: ()

The given CNF statement is satisfiable.

Assignments: 
	X = True
	Z = False

No. of branches:		 1

No. of unit propagations:	 1

No. of pure literals recursions: 1
